# Tracing 101

Step through this notebook to understand how tracing works in Generative AI Toolkit.

The Generative AI Toolkit comes with these tracers out-of-the-box:


In [1]:
from generative_ai_toolkit.tracer import (
    NoopTracer,
    HumanReadableTracer,
    InMemoryTracer,
    StructuredLogsTracer,
    TeeTracer,
)
from generative_ai_toolkit.tracer.dynamodb import DynamoDbTracer

import time
import random

### `InMemoryTracer`

Use the in-memory tracer for testing and development:


In [2]:
in_memory_tracer = InMemoryTracer(
    memory_size=1000  # Store max 1000 traces, before discarding older ones
)

# Context, added to all traces:
in_memory_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with in_memory_tracer.trace("parent") as parent_span:
    parent_span.add_attribute("foo", "bar")
    parent_span.add_attribute(
        "inherited.foo",
        "bar",
        inheritable=True,  # Inheritable attributes propagate to child spans
    )
    time.sleep(0.1)

    # Nested spans become child spans, that point to the parent (parent_span_id):
    with in_memory_tracer.trace("child") as child_span:
        child_span.add_attribute("bar", "foo")
        time.sleep(0.1)

for trace in in_memory_tracer.get_traces():
    print(trace)
    print()

Trace(span_name='parent', span_kind='INTERNAL', trace_id='6788ebce3b411a53f2c5d496dfef686c', span_id='c005c96483c8a738', parent_span_id=None, started_at=datetime.datetime(2025, 4, 15, 8, 26, 40, 789890, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 15, 8, 26, 40, 994665, tzinfo=datetime.timezone.utc), attributes={'foo': 'bar', 'inherited.foo': 'bar'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)

Trace(span_name='child', span_kind='INTERNAL', trace_id='6788ebce3b411a53f2c5d496dfef686c', span_id='d8088daf5d73dee9', parent_span_id='c005c96483c8a738', started_at=datetime.datetime(2025, 4, 15, 8, 26, 40, 892994, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 15, 8, 26, 40, 994640, tzinfo=datetime.timezone.utc), attributes={'bar': 'foo', 'inherited.foo': 'bar'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)



### Printing a human-readable version of traces during development

In the following example we add attributes that Generative AI Toolkit understands. It will use these to present traces in a way that is nicer to the human eye:


In [3]:
conversation_id = random.randint(0, 1000000)

with in_memory_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with in_memory_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in in_memory_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id  # filter traces by conversation id
    }
):
    print(trace.as_human_readable())
    print()

[c3e555cdaa25abc972f0ebff1315ac29/root/ae7ee58809dc613d] MyAgent SERVER 2025-04-15T08:26:41.001Z - parent (ai.conversation.id='150503' ai.auth.context='user123')


[c3e555cdaa25abc972f0ebff1315ac29/ae7ee58809dc613d/3b913b59a1af5cd4] MyAgent INTERNAL 2025-04-15T08:26:41.106Z - child (ai.trace.type='tool-invocation' ai.conversation.id='150503' ai.auth.context='user123')
       Input: Hello, world!
      Output: World, hello!




### `HumanReadableTracer`

You can also use the `HumanReadableTracer` that will log traces in human readable form to stdout, which is useful during development.

Note that traces are logged when the span ends, so parent spans are logged after child spans (this is true for all tracers):


In [4]:
import sys

human_readable_tracer = HumanReadableTracer(stream=sys.stdout)

human_readable_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with human_readable_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with human_readable_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

[dd51b6337050044f16089cddbd24ac27/d59c21d8e9ba7217/c18f84dce79045ac] MyAgent INTERNAL 2025-04-15T08:26:41.323Z - child (ai.trace.type='tool-invocation' ai.conversation.id='150503' ai.auth.context='user123')
       Input: Hello, world!
      Output: World, hello!

[dd51b6337050044f16089cddbd24ac27/root/d59c21d8e9ba7217] MyAgent SERVER 2025-04-15T08:26:41.218Z - parent (ai.conversation.id='150503' ai.auth.context='user123')



### `StructuredLogsTracer`

Use the `StructuredLogsTracer` to log traces to stdout as JSON:


In [5]:
structured_logs_tracer = StructuredLogsTracer(stream=sys.stdout)

structured_logs_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with structured_logs_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", "user123", inheritable=True)
    time.sleep(0.1)

    with structured_logs_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

{"logger":"TraceLogger","level":"INFO","message":"Trace","trace":{"span_name":"child","span_kind":"INTERNAL","trace_id":"f89f248bf66e714a2a226ff80f8f5a68","span_id":"047f0099476f696d","parent_span_id":"2f10627c38991983","started_at":"2025-04-15 08:26:41.543061+00:00","ended_at":"2025-04-15 08:26:41.648045+00:00","attributes":{"ai.trace.type":"tool-invocation","ai.tool.input":"Hello, world!","ai.tool.output":"World, hello!","ai.conversation.id":150503,"ai.auth.context":"user123"},"span_status":"UNSET","resource_attributes":{"service.name":"MyAgent"},"scope":{"name":"generative-ai-toolkit","version":"current"}}}
{"logger":"TraceLogger","level":"INFO","message":"Trace","trace":{"span_name":"parent","span_kind":"SERVER","trace_id":"f89f248bf66e714a2a226ff80f8f5a68","span_id":"2f10627c38991983","parent_span_id":null,"started_at":"2025-04-15 08:26:41.441425+00:00","ended_at":"2025-04-15 08:26:41.649877+00:00","attributes":{"ai.conversation.id":150503,"ai.auth.context":"user123"},"span_status

### `DynamoDbTracer`

Use the `DynamoDbTracer` to store traces to DynamoDB.

To use this tracer, you should have created a table with partition key `pk` (string) and sort key `sk` (string).

If you want to support getting traces by conversation ID, the table must have a GSI with partition key `conversation_id` (string) and sort key `sk` (string).

For example, here's how to create such a table:


In [6]:
!aws dynamodb create-table \
  --table-name MyTracesTable \
  --attribute-definitions \
    AttributeName=pk,AttributeType=S \
    AttributeName=sk,AttributeType=S \
    AttributeName=conversation_id,AttributeType=S \
  --key-schema \
    AttributeName=pk,KeyType=HASH \
    AttributeName=sk,KeyType=RANGE \
  --billing-mode PAY_PER_REQUEST \
  --global-secondary-indexes '[{"IndexName":"conversation_index","KeySchema":[{"AttributeName":"conversation_id","KeyType":"HASH"},{"AttributeName":"sk","KeyType":"RANGE"}],"Projection":{"ProjectionType":"ALL"}}]'


An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: MyTracesTable


Then, use that table in the `DynamoDbTracer`:


In [7]:
conversation_id = random.randint(0, 1000000)
auth_context = "user123"

ddb_tracer = DynamoDbTracer(
    table_name="MyTracesTable",
    identifier="MyAgent",
    conversation_id_gsi_name="conversation_index",
)

ddb_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with ddb_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", auth_context, inheritable=True)
    time.sleep(0.1)

    with ddb_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in ddb_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
    }
):
    print(trace.as_human_readable())
    print()

[ec083b0e69ec5347ed4c55358432e83e/root/a11894377f704b43] MyAgent SERVER 2025-04-15T08:26:43.105Z - parent (ai.conversation.id='583062' ai.auth.context='user123')


[ec083b0e69ec5347ed4c55358432e83e/a11894377f704b43/159dd6a16ce56149] MyAgent INTERNAL 2025-04-15T08:26:43.210Z - child (ai.trace.type='tool-invocation' ai.conversation.id='583062' ai.auth.context='user123')
       Input: Hello, world!
      Output: World, hello!




### `NoopTracer`

Use the no-operation tracer when you don't want traces:


In [8]:
noop_tracer = NoopTracer()
with noop_tracer.trace("noop") as span:
    span.add_attribute("foo", "bar")

# nothing was logged

### `TeeTracer`

Use the `TeeTracer` to send traces to multiple tracers at once.

Note that the first tracer you add, will be the one that `get_traces()` will be delegated to. So if you want to use that method, use a tracer that supports it.

Add tracers like this:


In [9]:
tee_tracer = TeeTracer()

# E.g. the DynamoDBTracer supports get_traces(), so add that first:
tee_tracer.add_tracer(ddb_tracer)

tee_tracer.add_tracer(human_readable_tracer)

# This is of course useless, but added for the sake of the example:
tee_tracer.add_tracer(noop_tracer)

Then, use the `TeeTracer` as any other tracer:


In [10]:
conversation_id = random.randint(0, 1000000)
auth_context = "user456"

tee_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with tee_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", auth_context, inheritable=True)
    time.sleep(0.1)

    with tee_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


print("==== from DynamoDB: ====")
for trace in tee_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
    }
):
    print(trace)
    print()

[5e639d61c8443a5011d72256c34f6634/41c00289ba6f00fc/03fd6f32f495abcf] MyAgent INTERNAL 2025-04-15T08:26:43.633Z - child (ai.trace.type='tool-invocation' ai.conversation.id='563083' ai.auth.context='user456')
       Input: Hello, world!
      Output: World, hello!

[5e639d61c8443a5011d72256c34f6634/root/41c00289ba6f00fc] MyAgent SERVER 2025-04-15T08:26:43.528Z - parent (ai.conversation.id='563083' ai.auth.context='user456')

==== from DynamoDB: ====
Trace(span_name='parent', span_kind='SERVER', trace_id='5e639d61c8443a5011d72256c34f6634', span_id='41c00289ba6f00fc', parent_span_id=None, started_at=datetime.datetime(2025, 4, 15, 8, 26, 43, 528401, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 15, 8, 26, 43, 764067, tzinfo=datetime.timezone.utc), attributes={'ai.conversation.id': 563083, 'ai.auth.context': 'user456'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)

Trace(span_name='child', span_kind='INTERNAL'